In [1]:
# Libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import datetime
import pickle
import math
import os

from tslearn.clustering import TimeSeriesKMeans
from tslearn.preprocessing import TimeSeriesScalerMinMax

import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cross_decomposition import PLSRegression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_predict
from sklearn.metrics import mean_absolute_error, mean_squared_error, pairwise_distances_argmin_min

from itertools import chain, combinations
from joblib import Parallel, delayed
from tabulate import tabulate

import warnings
warnings.filterwarnings('ignore')

In [2]:
def calculate_distortion(X_normalized, cluster_centers):
    cluster_centers_reduced = cluster_centers.reshape((cluster_centers.shape[0], -1))
    return sum(np.min(pairwise_distances_argmin_min(cluster_centers_reduced, X_normalized.reshape((X_normalized.shape[0], -1)), metric="euclidean"), axis=1))**2


def calculate_aic(y_true, y_pred, num_features):
    resid = y_true - y_pred
    sse = np.sum(resid ** 2)
    aic = len(y_true) * np.log(sse / len(y_true)) + 2 * num_features
    return aic


def calculate_bic(y_true, y_pred, num_features):
    resid = y_true - y_pred
    sse = np.sum(resid ** 2)
    n = len(y_true)
    bic = n * np.log(sse / n) + num_features * np.log(n)
    return bic


def forward_step(X_train, y_train, selected_features, remaining_features, criterion_func):
    best_criterion = np.inf
    best_feature = None
    temp_model = sm.OLS(y_train, sm.add_constant(X_train[selected_features])).fit()
    temp_predictions = temp_model.predict(sm.add_constant(X_train[selected_features]))
    for feature in remaining_features:
        temp_features = selected_features + [feature]
        temp_model = sm.OLS(y_train, sm.add_constant(X_train[temp_features])).fit()
        temp_criterion = criterion_func(y_train, temp_model.predict(sm.add_constant(X_train[temp_features])), len(temp_model.params))
        
        if temp_criterion < best_criterion:
            best_criterion = temp_criterion
            best_feature = feature
    
    return best_criterion, best_feature

def backward_step(X_train, y_train, selected_features, criterion_func):
    best_criterion = np.inf
    best_feature = None
    temp_model = sm.OLS(y_train, sm.add_constant(X_train[selected_features])).fit()
    temp_predictions = temp_model.predict(sm.add_constant(X_train[selected_features]))
    for feature in selected_features:
        temp_features = selected_features.copy()
        temp_features.remove(feature)
        temp_model = sm.OLS(y_train, sm.add_constant(X_train[temp_features])).fit()
        temp_criterion = criterion_func(y_train, temp_model.predict(sm.add_constant(X_train[temp_features])), len(temp_model.params))
        
        if temp_criterion < best_criterion:
            best_criterion = temp_criterion
            best_feature = feature
    
    return best_criterion, best_feature

def stepwise_bidirectional_selection(X_train, y_train, method='aic'):
    features = list(X_train.columns)
    selected_features = []
    best_criterion = np.inf
    
    while len(features) > 0:
        if method == 'aic':
            forward_criterion, forward_feature = forward_step(X_train, y_train, selected_features, features, calculate_aic)
            backward_criterion, backward_feature = backward_step(X_train, y_train, selected_features, calculate_aic)
        elif method == 'bic':
            forward_criterion, forward_feature = forward_step(X_train, y_train, selected_features, features, calculate_bic)
            backward_criterion, backward_feature = backward_step(X_train, y_train, selected_features, calculate_bic)
        else:
            raise ValueError("Invalid criterion_func. Use 'aic' or 'bic'.")
        
        if forward_criterion < backward_criterion and forward_criterion < best_criterion:
            selected_features.append(forward_feature)
            best_criterion = forward_criterion
        elif backward_criterion < forward_criterion and backward_criterion < best_criterion:
            selected_features.remove(backward_feature)
            best_criterion = backward_criterion
        else:
            break
    
    return selected_features


def r_squared(y_true, y_pred):
    mean_y_true = np.mean(y_true)
    ss_res = np.sum((y_true - y_pred) ** 2)
    ss_tot = np.sum((y_true - mean_y_true) ** 2)
    
    r_squared_value = 1 - (ss_res / ss_tot)
    
    return r_squared_value

def mape(y_true, y_pred):
    non_zero_indices = np.where(y_true != 0)[0]
    y_true_no_zeros = np.array(y_true)[non_zero_indices]
    y_pred_no_zeros = np.array(y_pred)[non_zero_indices]

    absolute_percentage_errors = np.abs((y_true_no_zeros - y_pred_no_zeros) / y_true_no_zeros)
    mape_value = np.mean(absolute_percentage_errors) * 100
    return mape_value

def mape_std(y_true, y_pred):
    mape_value = mape(y_true, y_pred)
    
    non_zero_indices = np.where(y_true != 0)[0]
    y_true_no_zeros = np.array(y_true)[non_zero_indices]
    y_pred_no_zeros = np.array(y_pred)[non_zero_indices]

    absolute_percentage_errors = np.abs((y_true_no_zeros - y_pred_no_zeros) / y_true_no_zeros)
    
    mape_sd_value = np.sqrt(np.mean((absolute_percentage_errors - mape_value / 100) ** 2))
    
    return mape_sd_value

def mae(y_true, y_pred):
    return mean_absolute_error(y_true, y_pred)

def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

def rmse_std(y_true, y_pred):
    return np.std(np.sqrt((y_true - y_pred) ** 2))


def optimize_ncomp_pls(X, y, n_comp_range, ylabel, objective):
    errors = []
    xticks = np.arange(1, n_comp_range + 1)

    for n_comp in xticks:
        pls = PLSRegression(n_components=n_comp)
        y_cv = cross_val_predict(pls, X, y, cv=10)
        error = rmse(y, y_cv)
        errors.append(error)

    if objective == 'min':
        return xticks[np.argmin(errors)]
    else:
        return xticks[np.argmax(errors)]


def show_figure(sensor_target_id,
                y_baseline,
                y_train,
                y_pred,
                y_test,
                y_predictions,
                irrigation_train,
                rain_train,
                irrigation_test,
                rain_test):

    all_data_train = np.concatenate([y_train, y_pred])
    all_data_test = np.concatenate([y_baseline, y_test, y_predictions])
    max_value_train = np.max(all_data_train)
    max_value_test = np.max(all_data_test)
    
    scaler = MinMaxScaler()
    
    irrigation_train_scaled = scaler.fit_transform(np.array(irrigation_train).reshape(-1, 1)).flatten()*max_value_train
    rain_train_scaled = scaler.fit_transform(np.array(rain_train).reshape(-1, 1)).flatten()*max_value_train
    irrigation_test_scaled = scaler.fit_transform(np.array(irrigation_test).reshape(-1, 1)).flatten()*max_value_test
    rain_test_scaled = scaler.fit_transform(np.array(rain_test).reshape(-1, 1)).flatten()*max_value_test

    fig = make_subplots(rows=2, cols=1, subplot_titles=('<b>Training</b>', '<b>Testing</b>'), vertical_spacing=0.1)
    fig.update_annotations(yshift=10)

    date_range = [datetime.datetime(2023, 5, 1) + datetime.timedelta(days=i) for i in range(len(y_test))]

    fig.add_trace(
        go.Scatter(x=[i for i in range(len(rain_train_scaled))], y=np.round(rain_train_scaled, 2),
            mode='lines',
            line=dict(color='cyan'),
            marker_size=3,
            marker_symbol='circle',
            name='Rain',
            showlegend=True,
            customdata=np.round(rain_train, 2),
            hovertemplate='Rain: %{customdata}<extra></extra>',
            marker_color='white',
            marker=dict(line=dict(color='cyan', width=2))
        ),
        row=1, col=1
    )

    fig.add_trace(
        go.Scatter(x=[i for i in range(len(irrigation_train_scaled))], y=np.round(irrigation_train_scaled, 2),
            mode='lines',
            line=dict(color='lime'),
            marker_size=3,
            marker_symbol='circle',
            name='Irrigation',
            showlegend=True,
            customdata=np.round(irrigation_train, 2),
            hovertemplate='Irrigation: %{customdata}<extra></extra>',
            marker_color='white',
            marker=dict(line=dict(color='lime', width=2))
        ),
        row=1, col=1
    )

    fig.add_trace(
        go.Scatter(x=[i for i in range(len(y_pred))], y=np.round(y_pred, 2),
            mode='lines',
            line=dict(color='green'),
            marker_size=3,
            marker_symbol='circle',
            name='Predictions',
            showlegend=True,
            marker_color='white',
            marker=dict(line=dict(color='green', width=2))
        ),
        row=1, col=1
    )

    fig.add_trace(
        go.Scatter(x=[i for i in range(len(y_train))], y=np.round(y_train, 2),
            mode='lines',
            line=dict(color='blue'),
            marker_size=3,
            marker_symbol='circle',
            name='Ground-truth',
            showlegend=True,
            marker_color='white',
            marker=dict(line=dict(color='blue', width=2))
        ),
        row=1, col=1
    )

    fig.add_trace(
        go.Scatter(x=[i for i in range(len(y_pred))], y=np.round(y_pred - y_train, 2),
            mode='markers',
            line=dict(color='red'),
            marker_size=3,
            marker_symbol='circle',
            name='Residuals',
            showlegend=True,
            marker_color='white',
            marker=dict(line=dict(color='red', width=2))
        ),
        row=1, col=1
    )

    fig.add_trace(
        go.Scatter(x=date_range, y=np.round(rain_test_scaled, 2),
            mode='lines',
            line=dict(color='cyan'),
            marker_size=3,
            marker_symbol='circle',
            name='Rain',
            showlegend=False,
            customdata=np.round(rain_test, 2),
            hovertemplate='Rain: %{customdata}<extra></extra>',
            marker_color='white',
            marker=dict(line=dict(color='cyan', width=2))
        ),
        row=2, col=1
    )

    fig.add_trace(
        go.Scatter(x=date_range, y=np.round(irrigation_test_scaled, 2),
            mode='lines',
            line=dict(color='lime'),
            marker_size=3,
            marker_symbol='circle',
            name='Irrigation',
            showlegend=False,
            customdata=np.round(irrigation_test, 2),
            hovertemplate='Irrigation: %{customdata}<extra></extra>',
            marker_color='white',
            marker=dict(line=dict(color='lime', width=2))
        ),
        row=2, col=1
    )

    fig.add_trace(
        go.Scatter(x=date_range, y=np.round(y_predictions, 2),
            mode='lines',
            line=dict(color='green'),
            marker_size=3,
            marker_symbol='circle',
            name='Predictions',
            showlegend=False,
            marker_color='white',
            marker=dict(line=dict(color='green', width=2))
        ),
        row=2, col=1
    )

    fig.add_trace(
        go.Scatter(x=date_range, y=np.round(y_test, 2),
            mode='lines',
            line=dict(color='blue'),
            marker_size=3,
            marker_symbol='circle',
            name='Ground-truth',
            showlegend=False,
            marker_color='white',
            marker=dict(line=dict(color='blue', width=2))
        ),
        row=2, col=1
    )

    fig.add_trace(
        go.Scatter(x=date_range, y=np.round(y_predictions - y_test, 2),
            mode='markers',
            line=dict(color='red'),
            marker_size=3,
            marker_symbol='circle',
            name='Residuals',
            showlegend=False,
            marker_color='white',
            marker=dict(line=dict(color='red', width=2))
        ),
        row=2, col=1
    )

    fig.add_trace(
        go.Scatter(x=date_range, y=np.round(y_baseline - y_test, 2),
            mode='markers',
            line=dict(color='orange'),
            marker_size=3,
            marker_symbol='circle',
            name='Residuals baseline',
            showlegend=True,
            marker_color='white',
            marker=dict(line=dict(color='orange', width=2))
        ),
        row=2, col=1
    )

    fig.update_layout(
        title=dict(
            text='<b>Sensor ' + str(sensor_target_id) + '</b>',
            font=dict(
                size=18 
            )
        ),
        autosize=False,
        width=2000,
        height=1000,
        margin=dict(
            l=50,
            r=50,
            b=100,
            t=100,
            pad=4
        )
    )
    fig.show()
    fig.write_image(os.path.join('plots', 'sensor_' + str(sensor_target_id) + '_plot.png'))  

In [3]:
df = pd.read_csv('data_sensors_rovere.csv')
df = df.rename(columns={'group': 'group_id'})

df_rovere = df[['reading_id', 'timestamp', 'sensor_id', 'value', 'description', 'group_id']]

df_rovere['reading_id'] = df_rovere['reading_id'].astype(str)
df_rovere['timestamp'] = pd.to_datetime(df_rovere['timestamp']).dt.floor('D').dt.date
df_rovere['sensor_id'] = df_rovere['sensor_id'].astype(str)
df_rovere['value'] = df_rovere['value'].astype(float)
df_rovere['description'] = df_rovere['description'].astype(str)
df_rovere['group_id'] = df_rovere['group_id'].astype(str)

tens_30 = ['72', '76', '73', '74', '61', '63', '67', '65']
tens_60 = ['71', '69', '75', '70', '62', '64', '68', '66']
tens_all = tens_30 + tens_60
    
condition_30 = df_rovere['sensor_id'].isin(tens_30)
condition_60 = df_rovere['sensor_id'].isin(tens_60)
condition_irrigation = df_rovere['description'] == 'irrigation'
    
df_rovere.loc[condition_30, 'description'] = 'Tensiometer 30'
df_rovere.loc[condition_60, 'description'] = 'Tensiometer 60'
df_rovere.loc[condition_irrigation, 'description'] = 'Irrigation'

condition_not_in_list = ~df_rovere['sensor_id'].isin(tens_all)
df_to_duplicate = df_rovere[condition_not_in_list]
df_to_duplicate['group_id_1'] = df_to_duplicate['group_id'] + '_1'
    
df_rovere = pd.concat([df_rovere, df_to_duplicate], ignore_index=True)
df_rovere.sort_values(by=['group_id', 'timestamp'], inplace=True)
df_rovere.reset_index(drop=True, inplace=True)

condition_group_id_1 = df_rovere['group_id_1'].notnull()
df_rovere.loc[condition_group_id_1, 'group_id'] = df_rovere.loc[condition_group_id_1, 'group_id_1']
df_rovere.drop(columns=['group_id_1'], inplace=True)

condition_update_group_id = df_rovere['sensor_id'].isin(tens_60)
df_rovere.loc[condition_update_group_id, 'group_id'] = df_rovere.loc[condition_update_group_id, 'group_id'] + '_1'


df_group = df_rovere.groupby(['timestamp', 'description', 'sensor_id', 'group_id']).agg({'value': ['min', 'max', 'mean', 'median', 'sum']}).reset_index()
df_group.columns = ['timestamp', 'description', 'sensor_id', 'group_id', 'val_min', 'val_max', 'val_avg', 'val_med', 'val_sum']

df_pivot = df_group.pivot(index=['timestamp', 'group_id'], columns='description', values=['val_min', 'val_max', 'val_avg', 'val_med', 'val_sum']).reset_index()
df_pivot.columns.name = None
df_pivot.columns = ['date', 'group_id', 'min_hum', 'min_temp', 'min_solar', 'min_wind', 'min_irr', 'min_rain', 'min_tens30', 'min_tens60',
                    'max_hum', 'max_temp', 'max_solar', 'max_wind', 'max_irr', 'max_rain', 'max_tens30', 'max_tens60',
                    'avg_hum', 'avg_temp', 'avg_solar', 'avg_wind', 'avg_irr', 'avg_rain', 'avg_tens30', 'avg_tens60',
                    'med_hum', 'med_temp', 'med_solar', 'med_wind', 'med_irr', 'med_rain', 'med_tens30', 'med_tens60',
                    'sum_hum', 'sum_temp', 'sum_solar', 'sum_wind', 'sum_irr', 'sum_rain', 'sum_tens30', 'sum_tens60']

df_pivot['min_tens'] = df_pivot['min_tens30'].combine_first(df_pivot['min_tens60'])
df_pivot.drop(columns=['min_tens30', 'min_tens60'], inplace=True)


df_pivot['max_tens'] = df_pivot['max_tens30'].combine_first(df_pivot['max_tens60'])
df_pivot.drop(columns=['max_tens30', 'max_tens60'], inplace=True)


df_pivot['avg_tens'] = df_pivot['avg_tens30'].combine_first(df_pivot['avg_tens60'])
df_pivot.drop(columns=['avg_tens30', 'avg_tens60'], inplace=True)


df_pivot['med_tens'] = df_pivot['med_tens30'].combine_first(df_pivot['med_tens60'])
df_pivot.drop(columns=['med_tens30', 'med_tens60'], inplace=True)


df_pivot['sum_tens'] = df_pivot['sum_tens30'].combine_first(df_pivot['sum_tens60'])
df_pivot.drop(columns=['sum_tens30', 'sum_tens60'], inplace=True)

df_pivot = df_pivot.reset_index(drop=True)

df = df_pivot
df

In [ ]:
min_date = df['date'].min()
max_date = df['date'].max()

all_dates = pd.date_range(start=min_date, end=max_date)
existing_dates_set = set(df['date'])

missing_dates = [date.date() for date in all_dates if date.date() not in existing_dates_set]
missing_dates

In [ ]:
# repeated_dates = np.tile(missing_dates, 16)

# sensor_ids = df['group_id'].unique()
# sensor_ids = np.repeat(sensor_ids, 3)

# missing_data = pd.DataFrame({'date': repeated_dates, 'group_id': sensor_ids})

# df = df.merge(missing_data, on=['date', 'group_id'], how='outer')
# df = pd.DataFrame(df)
# df = df.sort_values(by=['group_id', 'date']).reset_index(drop=True)
# df

In [ ]:
num_missing_values = df.isna().any(axis=1).sum()
print("Numero di righe con valori mancanti:", num_missing_values)

In [ ]:
# float_columns = df.select_dtypes(include=['float']).columns
# df[float_columns] = df[float_columns].interpolate(method='linear', limit_direction='both')

# num_missing_values = df.isna().any(axis=1).sum()
# print("Numero di righe con valori mancanti:", num_missing_values)

In [ ]:
columns_to_drop = ['min_irr', 'max_irr', 'avg_irr', 'med_irr', 'min_rain', 'avg_rain', 'sum_hum',
                   'sum_temp', 'sum_solar', 'min_wind', 'max_wind', 'avg_wind', 'sum_wind', 'med_wind']
df = df.drop(columns=columns_to_drop).dropna().reset_index(drop=True)


group_id_mapping = {
    '1': '72',
    '2': '76',
    '3': '73',
    '4': '74',
    '5': '61',
    '6': '63',
    '7': '67',
    '8': '65',
    '1_1': '71',
    '2_1': '69',
    '3_1': '75',
    '4_1': '70',
    '5_1': '62',
    '6_1': '64',
    '7_1': '68',
    '8_1': '66'
}

df['group_id'] = df['group_id'].replace(group_id_mapping)
df = df.rename(columns={'group_id': 'sensor_id'})
df = df[['sensor_id', 'date', 'avg_tens'] + [col for col in df.columns if col not in ['sensor_id', 'date', 'avg_tens']]]
df = df.sort_values(by=['sensor_id', 'date']).reset_index(drop=True)

ids = df['sensor_id']
dates = df['date']
X = df.drop(columns=['date', 'sensor_id'])
X = X.shift(1).add_suffix('_lag1').join(X.shift(2).add_suffix('_lag2')).join(X.shift(3).add_suffix('_lag3'))

X['date'] = dates
dates_to_remove = [datetime.date(2023, 4, 28), datetime.date(2023, 4, 29), datetime.date(2023, 4, 30)]
X = X[~X['date'].isin(dates_to_remove)].reset_index(drop=True)
X = X.drop(columns='date')

y = df[['sensor_id', 'date', 'avg_tens']]
y = y[~y['date'].isin(dates_to_remove)].reset_index(drop=True)

df_merged = pd.concat([y, X], axis=1)
df_merged

In [4]:
tens_combined = list(set(tens_all))
tens_ordered = sorted(tens_combined, key=lambda x: int(x))


df_cluster = df[['date', 'sensor_id', 'avg_tens']]
df_transformed = pd.pivot_table(df_cluster, values='avg_tens', index='date', columns='sensor_id', aggfunc='mean').reset_index()
data_array = np.array(df_transformed.T.drop('date').values)

X_normalized = TimeSeriesScalerMinMax().fit_transform(data_array)
X_flattened = X_normalized.reshape((X_normalized.shape[0], -1))

n_clusters_range = range(2, 10)
num_executions = 10
np.random.seed(0)

average_distortions = []

for n_clusters in n_clusters_range:
    distortions_for_cluster = []

    for _ in range(num_executions):

        seed = np.random.randint(0, 1000)
        np.random.seed(seed)

        model = TimeSeriesKMeans(n_clusters=n_clusters, metric='dtw', max_iter=10)
        model.fit(X_normalized)
        cluster_centers = model.cluster_centers_
        distortion = calculate_distortion(X_flattened, cluster_centers)
        distortions_for_cluster.append(distortion)

    average_distortion = np.mean(distortions_for_cluster)
    average_distortions.append(average_distortion)


plt.plot(n_clusters_range, average_distortions, marker='o', label='DTW Clustering')
plt.xlabel('Number of Clusters')
plt.ylabel('Distance from Centroids')
plt.title('Select the Optimal Number of Clusters')
plt.legend()
plt.show()

In [10]:
model_1 = TimeSeriesKMeans(n_clusters=3, metric='dtw', max_iter=100)
model_1.fit(data_array)
clusters_1=model_1.predict(data_array)

model_2 = TimeSeriesKMeans(n_clusters=6, metric='dtw', max_iter=100)
model_2.fit(data_array)
clusters_2=model_2.predict(data_array)

df_cluster = pd.DataFrame({'Sensor_ID': tens_ordered, 'Cluster 1': clusters_1, 'Cluster 2': clusters_2})
df_cluster

In [11]:
sensor_ids = ['72', '76', '73', '74', '61', '63', '67', '65', '71', '69', '75', '70', '62', '64', '68', '66']
sensor_ids.sort()

residuals = []

for sensor in sensor_ids:
    
    subset = df_merged[df_merged['sensor_id'] == sensor].copy()
    X = subset.drop(['sensor_id', 'date', 'avg_tens', 'avg_tens_lag1', 'avg_tens_lag2', 'avg_tens_lag3'], axis=1)
    y = subset['avg_tens']
    
    selected_features = stepwise_bidirectional_selection(X, y, method='aic')
    model = sm.OLS(y, sm.add_constant(X[selected_features])).fit()

    intercept = model.params[0]
    intercept_vector = np.full(3, intercept)

    pred_sensor = model.predict(sm.add_constant(X[selected_features]))
    combined_vector = np.concatenate((intercept_vector, pred_sensor))

    selected_values = df.loc[df['sensor_id'] == sensor, 'avg_tens']
    values = selected_values - combined_vector
    res = list(zip(values))
    
    residuals.extend(res)


df_residuals = pd.DataFrame(residuals, columns=['residuals'])

df_residuals = df_residuals.shift(1).add_suffix('_lag1').join(df_residuals.shift(2).add_suffix('_lag2')).join(df_residuals.shift(3).add_suffix('_lag3'))
df_residuals = pd.concat([df['date'], df_residuals], axis=1)

dates_to_remove = [datetime.date(2023, 4, 28), datetime.date(2023, 4, 29), datetime.date(2023, 4, 30)]
df_residuals = df_residuals[~df_residuals['date'].isin(dates_to_remove)].reset_index(drop=True)
df_residuals = df_residuals.drop(columns='date')

data = pd.concat([df_merged, df_residuals], axis=1)
data = data.drop(columns='date')
data

# Main Sensors (ARIMAX top results)

These are the sensors among those we focused on the most where the ARIMAX models performed the best in terms of prediction error.

## Sensor 64

In [12]:
best_subset = ['65', '66', '69', '74']

df_train = data[data['sensor_id'].isin(list(best_subset))].reset_index(drop=True)
df_test = data[data['sensor_id'] == '64'].reset_index(drop=True)

X_train = df_train.drop(['sensor_id', 'avg_tens'], axis=1).reset_index(drop=True)
y_train = df_train['avg_tens'].reset_index(drop=True)

X_test = df_test.drop(['sensor_id', 'avg_tens'], axis=1).reset_index(drop=True)
y_test = df_test['avg_tens'].reset_index(drop=True)

selected_features = stepwise_bidirectional_selection(X_train, y_train, method='aic')

model = sm.OLS(y_train, sm.add_constant(X_train[selected_features])).fit()
pred_sensor = model.predict(sm.add_constant(X_test[selected_features])).reset_index(drop=True)


y_baseline = df_test['avg_tens_lag1'].reset_index(drop=True)
y_train_plot = y_train.reset_index(drop=True)
pred_train = model.predict(sm.add_constant(X_train[selected_features])).reset_index(drop=True)

y_test_plot = y_test.reset_index(drop=True)
pred_test = pred_sensor.reset_index(drop=True)

irrigation_train = df[df['sensor_id'].isin(list(best_subset))][['date', 'sum_irr']]
rain_train = df[df['sensor_id'].isin(list(best_subset))][['date', 'sum_rain']]

dates_to_remove = [datetime.date(2023, 4, 28), datetime.date(2023, 4, 29), datetime.date(2023, 4, 30)]
irrigation_train = irrigation_train[~irrigation_train['date'].isin(dates_to_remove)].reset_index(drop=True)
rain_train = rain_train[~rain_train['date'].isin(dates_to_remove)].reset_index(drop=True)

irrigation_train = irrigation_train['sum_irr']
rain_train = rain_train['sum_rain']

irrigation_test = df[df['sensor_id'] == '64'][['date', 'sum_irr']].reset_index(drop=True)
rain_test = df[df['sensor_id'] == '64'][['date', 'sum_rain']].reset_index(drop=True)

irrigation_test = irrigation_test[~irrigation_test['date'].isin(dates_to_remove)].reset_index(drop=True)
rain_test = rain_test[~rain_test['date'].isin(dates_to_remove)].reset_index(drop=True)

irrigation_test = irrigation_test['sum_irr']
rain_test = rain_test['sum_rain']


selected_features

In [13]:
r_squared_arimax = round(r_squared(y_test, pred_sensor), 3)
mape_arimax = round(mape(y_test, pred_sensor), 3)
mape_std_arimax = round(mape_std(y_test, pred_sensor), 3)
mae_arimax = round(mae(y_test, pred_sensor), 3)
rmse_arimax = round(rmse(y_test, pred_sensor), 3)
rmse_std_arimax = round(rmse_std(y_test, pred_sensor), 3)


table = [
    ['Metric', 'ARIMAX'],
    ['R-squared', r_squared_arimax],
    ['MAPE', mape_arimax],
    ['MAPE_std', mape_std_arimax],
    ['MAE', mae_arimax],
    ['RMSE', rmse_arimax],
    ['RMSE_std', rmse_std_arimax]
]

print(tabulate(table, headers='firstrow', tablefmt='fancy_grid'))

In [14]:
model.summary()

In [37]:
show_figure(sensor_target_id=64,
            y_baseline=y_baseline,
            y_train=y_train_plot,
            y_pred=pred_train,
            y_test=y_test_plot,
            y_predictions=pred_test,
            irrigation_train=irrigation_train,
            rain_train=rain_train,
            irrigation_test=irrigation_test,
            rain_test=rain_test)

In [38]:
file_path = os.path.join('predictions', 'sensor_64.pickle')

with open(file_path, 'wb') as file:
    pickle.dump(pred_sensor, file)

## Sensor 61

In [39]:
best_subset = ['62', '70', '71', '75']

df_train = data[data['sensor_id'].isin(list(best_subset))].reset_index(drop=True)
df_test = data[data['sensor_id'] == '61'].reset_index(drop=True)

X_train = df_train.drop(['sensor_id', 'avg_tens'], axis=1).reset_index(drop=True)
y_train = df_train['avg_tens'].reset_index(drop=True)

X_test = df_test.drop(['sensor_id', 'avg_tens'], axis=1).reset_index(drop=True)
y_test = df_test['avg_tens'].reset_index(drop=True)

selected_features = stepwise_bidirectional_selection(X_train, y_train, method='aic')

model = sm.OLS(y_train, sm.add_constant(X_train[selected_features])).fit()
pred_sensor = model.predict(sm.add_constant(X_test[selected_features])).reset_index(drop=True)


y_baseline = df_test['avg_tens_lag1'].reset_index(drop=True)
y_train_plot = y_train.reset_index(drop=True)
pred_train = model.predict(sm.add_constant(X_train[selected_features])).reset_index(drop=True)

y_test_plot = y_test.reset_index(drop=True)
pred_test = pred_sensor.reset_index(drop=True)

irrigation_train = df[df['sensor_id'].isin(list(best_subset))][['date', 'sum_irr']]
rain_train = df[df['sensor_id'].isin(list(best_subset))][['date', 'sum_rain']]

dates_to_remove = [datetime.date(2023, 4, 28), datetime.date(2023, 4, 29), datetime.date(2023, 4, 30)]
irrigation_train = irrigation_train[~irrigation_train['date'].isin(dates_to_remove)].reset_index(drop=True)
rain_train = rain_train[~rain_train['date'].isin(dates_to_remove)].reset_index(drop=True)

irrigation_train = irrigation_train['sum_irr']
rain_train = rain_train['sum_rain']

irrigation_test = df[df['sensor_id'] == '61'][['date', 'sum_irr']].reset_index(drop=True)
rain_test = df[df['sensor_id'] == '61'][['date', 'sum_rain']].reset_index(drop=True)

irrigation_test = irrigation_test[~irrigation_test['date'].isin(dates_to_remove)].reset_index(drop=True)
rain_test = rain_test[~rain_test['date'].isin(dates_to_remove)].reset_index(drop=True)

irrigation_test = irrigation_test['sum_irr']
rain_test = rain_test['sum_rain']

In [40]:
r_squared_arimax = round(r_squared(y_test, pred_sensor), 3)
mape_arimax = round(mape(y_test, pred_sensor), 3)
mape_std_arimax = round(mape_std(y_test, pred_sensor), 3)
mae_arimax = round(mae(y_test, pred_sensor), 3)
rmse_arimax = round(rmse(y_test, pred_sensor), 3)
rmse_std_arimax = round(rmse_std(y_test, pred_sensor), 3)


table = [
    ['Metric', 'ARIMAX'],
    ['R-squared', r_squared_arimax],
    ['MAPE', mape_arimax],
    ['MAPE_std', mape_std_arimax],
    ['MAE', mae_arimax],
    ['RMSE', rmse_arimax],
    ['RMSE_std', rmse_std_arimax]
]

print(tabulate(table, headers='firstrow', tablefmt='fancy_grid'))

In [41]:
show_figure(sensor_target_id=61,
            y_baseline=y_baseline,
            y_train=y_train_plot,
            y_pred=pred_train,
            y_test=y_test_plot,
            y_predictions=pred_test,
            irrigation_train=irrigation_train,
            rain_train=rain_train,
            irrigation_test=irrigation_test,
            rain_test=rain_test)

In [42]:
file_path = os.path.join('predictions', 'sensor_61.pickle')

with open(file_path, 'wb') as file:
    pickle.dump(pred_sensor, file)

## Sensor 70

In [47]:
best_subset = ['64', '65', '68', '69', '71']

df_train = data[data['sensor_id'].isin(list(best_subset))]
df_test = data[data['sensor_id'] == '70']

X_train = df_train.drop(['sensor_id', 'avg_tens'], axis=1)
y_train = df_train['avg_tens']

X_test = df_test.drop(['sensor_id', 'avg_tens'], axis=1)
y_test = df_test['avg_tens']

selected_features = stepwise_bidirectional_selection(X_train, y_train, method='aic')

model = sm.OLS(y_train, sm.add_constant(X_train[selected_features])).fit()
pred_sensor = model.predict(sm.add_constant(X_test[selected_features]))


y_baseline = df_test['avg_tens_lag1'].reset_index(drop=True)

y_train_plot = y_train.reset_index(drop=True)
pred_train = model.predict(sm.add_constant(X_train[selected_features])).reset_index(drop=True)

y_test_plot = y_test.reset_index(drop=True)
pred_test = pred_sensor.reset_index(drop=True)

irrigation_train = df[df['sensor_id'].isin(list(best_subset))][['date', 'sum_irr']]
rain_train = df[df['sensor_id'].isin(list(best_subset))][['date', 'sum_rain']]

dates_to_remove = [datetime.date(2023, 4, 28), datetime.date(2023, 4, 29), datetime.date(2023, 4, 30)]
irrigation_train = irrigation_train[~irrigation_train['date'].isin(dates_to_remove)].reset_index(drop=True)
rain_train = rain_train[~rain_train['date'].isin(dates_to_remove)].reset_index(drop=True)

irrigation_train = irrigation_train['sum_irr']
rain_train = rain_train['sum_rain']

irrigation_test = df[df['sensor_id'] == '70'][['date', 'sum_irr']].reset_index(drop=True)
rain_test = df[df['sensor_id'] == '70'][['date', 'sum_rain']].reset_index(drop=True)

irrigation_test = irrigation_test[~irrigation_test['date'].isin(dates_to_remove)].reset_index(drop=True)
rain_test = rain_test[~rain_test['date'].isin(dates_to_remove)].reset_index(drop=True)

irrigation_test = irrigation_test['sum_irr']
rain_test = rain_test['sum_rain']


selected_features

In [48]:
r_squared_arimax = round(r_squared(y_test, pred_sensor), 3)
mape_arimax = round(mape(y_test, pred_sensor), 3)
mape_std_arimax = round(mape_std(y_test, pred_sensor), 3)
mae_arimax = round(mae(y_test, pred_sensor), 3)
rmse_arimax = round(rmse(y_test, pred_sensor), 3)
rmse_std_arimax = round(rmse_std(y_test, pred_sensor), 3)


table = [
    ['Metric', 'ARIMAX'],
    ['R-squared', r_squared_arimax],
    ['MAPE', mape_arimax],
    ['MAPE_std', mape_std_arimax],
    ['MAE', mae_arimax],
    ['RMSE', rmse_arimax],
    ['RMSE_std', rmse_std_arimax]
]

print(tabulate(table, headers='firstrow', tablefmt='fancy_grid'))

In [49]:
show_figure(sensor_target_id=70,
            y_baseline=y_baseline,
            y_train=y_train_plot,
            y_pred=pred_train,
            y_test=y_test_plot,
            y_predictions=pred_test,
            irrigation_train=irrigation_train,
            rain_train=rain_train,
            irrigation_test=irrigation_test,
            rain_test=rain_test)

In [50]:
file_path = os.path.join('predictions', 'sensor_70.pickle')

with open(file_path, 'wb') as file:
    pickle.dump(pred_sensor, file)

# Main Sensors (ARIMAX is not the best)

These are the sensors among those we focused on most where the ARIMAX models did not perform the best in terms of prediction error when compared to other models.

## Sensor 62

In [51]:
best_subset = ['61', '64']

df_train = data[data['sensor_id'].isin(list(best_subset))]
df_test = data[data['sensor_id'] == '62']

X_train = df_train.drop(['sensor_id', 'avg_tens'], axis=1)
y_train = df_train['avg_tens']

X_test = df_test.drop(['sensor_id', 'avg_tens'], axis=1)
y_test = df_test['avg_tens']

selected_features = stepwise_bidirectional_selection(X_train, y_train, method='aic')

model = sm.OLS(y_train, sm.add_constant(X_train[selected_features])).fit()
pred_sensor = model.predict(sm.add_constant(X_test[selected_features]))


y_baseline = df_test['avg_tens_lag1'].reset_index(drop=True)
y_train_plot = y_train.reset_index(drop=True)
pred_train = model.predict(sm.add_constant(X_train[selected_features])).reset_index(drop=True)

y_test_plot = y_test.reset_index(drop=True)
pred_test = pred_sensor.reset_index(drop=True)

irrigation_train = df[df['sensor_id'].isin(list(best_subset))][['date', 'sum_irr']]
rain_train = df[df['sensor_id'].isin(list(best_subset))][['date', 'sum_rain']]

dates_to_remove = [datetime.date(2023, 4, 28), datetime.date(2023, 4, 29), datetime.date(2023, 4, 30)]
irrigation_train = irrigation_train[~irrigation_train['date'].isin(dates_to_remove)].reset_index(drop=True)
rain_train = rain_train[~rain_train['date'].isin(dates_to_remove)].reset_index(drop=True)

irrigation_train = irrigation_train['sum_irr']
rain_train = rain_train['sum_rain']

irrigation_test = df[df['sensor_id'] == '62'][['date', 'sum_irr']].reset_index(drop=True)
rain_test = df[df['sensor_id'] == '62'][['date', 'sum_rain']].reset_index(drop=True)

irrigation_test = irrigation_test[~irrigation_test['date'].isin(dates_to_remove)].reset_index(drop=True)
rain_test = rain_test[~rain_test['date'].isin(dates_to_remove)].reset_index(drop=True)

irrigation_test = irrigation_test['sum_irr']
rain_test = rain_test['sum_rain']


selected_features

In [52]:
r_squared_arimax = round(r_squared(y_test, pred_sensor), 3)
mape_arimax = round(mape(y_test, pred_sensor), 3)
mape_std_arimax = round(mape_std(y_test, pred_sensor), 3)
mae_arimax = round(mae(y_test, pred_sensor), 3)
rmse_arimax = round(rmse(y_test, pred_sensor), 3)
rmse_std_arimax = round(rmse_std(y_test, pred_sensor), 3)


table = [
    ['Metric', 'ARIMAX'],
    ['R-squared', r_squared_arimax],
    ['MAPE', mape_arimax],
    ['MAPE_std', mape_std_arimax],
    ['MAE', mae_arimax],
    ['RMSE', rmse_arimax],
    ['RMSE_std', rmse_std_arimax]
]

print(tabulate(table, headers='firstrow', tablefmt='fancy_grid'))

In [53]:
show_figure(sensor_target_id=62,
            y_baseline=y_baseline,
            y_train=y_train_plot,
            y_pred=pred_train,
            y_test=y_test_plot,
            y_predictions=pred_test,
            irrigation_train=irrigation_train,
            rain_train=rain_train,
            irrigation_test=irrigation_test,
            rain_test=rain_test)

In [54]:
file_path = os.path.join('predictions', 'sensor_62.pickle')

with open(file_path, 'wb') as file:
    pickle.dump(pred_sensor, file)

## Sensor 63

In [58]:
best_subset = ['64', '67']

df_train = data[data['sensor_id'].isin(list(best_subset))]
df_test = data[data['sensor_id'] == '63']

X_train = df_train.drop(['sensor_id', 'avg_tens'], axis=1)
y_train = df_train['avg_tens']

X_test = df_test.drop(['sensor_id', 'avg_tens'], axis=1)
y_test = df_test['avg_tens']

selected_features = stepwise_bidirectional_selection(X_train, y_train, method='aic')

model = sm.OLS(y_train, sm.add_constant(X_train[selected_features])).fit()
pred_sensor = model.predict(sm.add_constant(X_test[selected_features]))


y_baseline = df_test['avg_tens_lag1'].reset_index(drop=True)
y_train_plot = y_train.reset_index(drop=True)
pred_train = model.predict(sm.add_constant(X_train[selected_features])).reset_index(drop=True)

y_test_plot = y_test.reset_index(drop=True)
pred_test = pred_sensor.reset_index(drop=True)

irrigation_train = df[df['sensor_id'].isin(list(best_subset))][['date', 'sum_irr']]
rain_train = df[df['sensor_id'].isin(list(best_subset))][['date', 'sum_rain']]

dates_to_remove = [datetime.date(2023, 4, 28), datetime.date(2023, 4, 29), datetime.date(2023, 4, 30)]
irrigation_train = irrigation_train[~irrigation_train['date'].isin(dates_to_remove)].reset_index(drop=True)
rain_train = rain_train[~rain_train['date'].isin(dates_to_remove)].reset_index(drop=True)

irrigation_train = irrigation_train['sum_irr']
rain_train = rain_train['sum_rain']

irrigation_test = df[df['sensor_id'] == '63'][['date', 'sum_irr']].reset_index(drop=True)
rain_test = df[df['sensor_id'] == '63'][['date', 'sum_rain']].reset_index(drop=True)

irrigation_test = irrigation_test[~irrigation_test['date'].isin(dates_to_remove)].reset_index(drop=True)
rain_test = rain_test[~rain_test['date'].isin(dates_to_remove)].reset_index(drop=True)

irrigation_test = irrigation_test['sum_irr']
rain_test = rain_test['sum_rain']


selected_features

In [59]:
r_squared_arimax = round(r_squared(y_test, pred_sensor), 3)
mape_arimax = round(mape(y_test, pred_sensor), 3)
mape_std_arimax = round(mape_std(y_test, pred_sensor), 3)
mae_arimax = round(mae(y_test, pred_sensor), 3)
rmse_arimax = round(rmse(y_test, pred_sensor), 3)
rmse_std_arimax = round(rmse_std(y_test, pred_sensor), 3)


table = [
    ['Metric', 'ARIMAX'],
    ['R-squared', r_squared_arimax],
    ['MAPE', mape_arimax],
    ['MAPE_std', mape_std_arimax],
    ['MAE', mae_arimax],
    ['RMSE', rmse_arimax],
    ['RMSE_std', rmse_std_arimax]
]

print(tabulate(table, headers='firstrow', tablefmt='fancy_grid'))

In [60]:
show_figure(sensor_target_id=63,
            y_baseline=y_baseline,
            y_train=y_train_plot,
            y_pred=pred_train,
            y_test=y_test_plot,
            y_predictions=pred_test,
            irrigation_train=irrigation_train,
            rain_train=rain_train,
            irrigation_test=irrigation_test,
            rain_test=rain_test)

In [61]:
file_path = os.path.join('predictions', 'sensor_63.pickle')

with open(file_path, 'wb') as file:
    pickle.dump(pred_sensor, file)

## Sensor 67

In [63]:
best_subset = ['68']

df_train = data[data['sensor_id'].isin(list(best_subset))]
df_test = data[data['sensor_id'] == '67']

X_train = df_train.drop(['sensor_id', 'avg_tens'], axis=1)
y_train = df_train['avg_tens']

X_test = df_test.drop(['sensor_id', 'avg_tens'], axis=1)
y_test = df_test['avg_tens']

selected_features = stepwise_bidirectional_selection(X_train, y_train, method='aic')

model = sm.OLS(y_train, sm.add_constant(X_train[selected_features])).fit()
pred_sensor = model.predict(sm.add_constant(X_test[selected_features]))


y_baseline = df_test['avg_tens_lag1'].reset_index(drop=True)
y_train_plot = y_train.reset_index(drop=True)
pred_train = model.predict(sm.add_constant(X_train[selected_features])).reset_index(drop=True)

y_test_plot = y_test.reset_index(drop=True)
pred_test = pred_sensor.reset_index(drop=True)

irrigation_train = df[df['sensor_id'].isin(list(best_subset))][['date', 'sum_irr']]
rain_train = df[df['sensor_id'].isin(list(best_subset))][['date', 'sum_rain']]

dates_to_remove = [datetime.date(2023, 4, 28), datetime.date(2023, 4, 29), datetime.date(2023, 4, 30)]
irrigation_train = irrigation_train[~irrigation_train['date'].isin(dates_to_remove)].reset_index(drop=True)
rain_train = rain_train[~rain_train['date'].isin(dates_to_remove)].reset_index(drop=True)

irrigation_train = irrigation_train['sum_irr']
rain_train = rain_train['sum_rain']

irrigation_test = df[df['sensor_id'] == '67'][['date', 'sum_irr']].reset_index(drop=True)
rain_test = df[df['sensor_id'] == '67'][['date', 'sum_rain']].reset_index(drop=True)

irrigation_test = irrigation_test[~irrigation_test['date'].isin(dates_to_remove)].reset_index(drop=True)
rain_test = rain_test[~rain_test['date'].isin(dates_to_remove)].reset_index(drop=True)

irrigation_test = irrigation_test['sum_irr']
rain_test = rain_test['sum_rain']


selected_features

In [64]:
r_squared_arimax = round(r_squared(y_test, pred_sensor), 3)
mape_arimax = round(mape(y_test, pred_sensor), 3)
mape_std_arimax = round(mape_std(y_test, pred_sensor), 3)
mae_arimax = round(mae(y_test, pred_sensor), 3)
rmse_arimax = round(rmse(y_test, pred_sensor), 3)
rmse_std_arimax = round(rmse_std(y_test, pred_sensor), 3)


table = [
    ['Metric', 'ARIMAX'],
    ['R-squared', r_squared_arimax],
    ['MAPE', mape_arimax],
    ['MAPE_std', mape_std_arimax],
    ['MAE', mae_arimax],
    ['RMSE', rmse_arimax],
    ['RMSE_std', rmse_std_arimax]
]

print(tabulate(table, headers='firstrow', tablefmt='fancy_grid'))

In [65]:
show_figure(sensor_target_id=67,
            y_baseline=y_baseline,
            y_train=y_train_plot,
            y_pred=pred_train,
            y_test=y_test_plot,
            y_predictions=pred_test,
            irrigation_train=irrigation_train,
            rain_train=rain_train,
            irrigation_test=irrigation_test,
            rain_test=rain_test)

In [66]:
file_path = os.path.join('predictions', 'sensor_67.pickle')

with open(file_path, 'wb') as file:
    pickle.dump(pred_sensor, file)

## Sensor 68

In [67]:
best_subset = ['67']

df_train = data[data['sensor_id'].isin(list(best_subset))]
df_test = data[data['sensor_id'] == '68']

X_train = df_train.drop(['sensor_id', 'avg_tens'], axis=1)
y_train = df_train['avg_tens']

X_test = df_test.drop(['sensor_id', 'avg_tens'], axis=1)
y_test = df_test['avg_tens']

selected_features = stepwise_bidirectional_selection(X_train, y_train, method='aic')

model = sm.OLS(y_train, sm.add_constant(X_train[selected_features])).fit()
pred_sensor = model.predict(sm.add_constant(X_test[selected_features]))


y_baseline = df_test['avg_tens_lag1'].reset_index(drop=True)
y_train_plot = y_train.reset_index(drop=True)
pred_train = model.predict(sm.add_constant(X_train[selected_features])).reset_index(drop=True)

y_test_plot = y_test.reset_index(drop=True)
pred_test = pred_sensor.reset_index(drop=True)

irrigation_train = df[df['sensor_id'].isin(list(best_subset))][['date', 'sum_irr']]
rain_train = df[df['sensor_id'].isin(list(best_subset))][['date', 'sum_rain']]

dates_to_remove = [datetime.date(2023, 4, 28), datetime.date(2023, 4, 29), datetime.date(2023, 4, 30)]
irrigation_train = irrigation_train[~irrigation_train['date'].isin(dates_to_remove)].reset_index(drop=True)
rain_train = rain_train[~rain_train['date'].isin(dates_to_remove)].reset_index(drop=True)

irrigation_train = irrigation_train['sum_irr']
rain_train = rain_train['sum_rain']

irrigation_test = df[df['sensor_id'] == '68'][['date', 'sum_irr']].reset_index(drop=True)
rain_test = df[df['sensor_id'] == '68'][['date', 'sum_rain']].reset_index(drop=True)

irrigation_test = irrigation_test[~irrigation_test['date'].isin(dates_to_remove)].reset_index(drop=True)
rain_test = rain_test[~rain_test['date'].isin(dates_to_remove)].reset_index(drop=True)

irrigation_test = irrigation_test['sum_irr']
rain_test = rain_test['sum_rain']


selected_features

In [68]:
r_squared_arimax = round(r_squared(y_test, pred_sensor), 3)
mape_arimax = round(mape(y_test, pred_sensor), 3)
mape_std_arimax = round(mape_std(y_test, pred_sensor), 3)
mae_arimax = round(mae(y_test, pred_sensor), 3)
rmse_arimax = round(rmse(y_test, pred_sensor), 3)
rmse_std_arimax = round(rmse_std(y_test, pred_sensor), 3)


table = [
    ['Metric', 'ARIMAX'],
    ['R-squared', r_squared_arimax],
    ['MAPE', mape_arimax],
    ['MAPE_std', mape_std_arimax],
    ['MAE', mae_arimax],
    ['RMSE', rmse_arimax],
    ['RMSE_std', rmse_std_arimax]
]

print(tabulate(table, headers='firstrow', tablefmt='fancy_grid'))

In [69]:
show_figure(sensor_target_id=68,
            y_baseline=y_baseline,
            y_train=y_train_plot,
            y_pred=pred_train,
            y_test=y_test_plot,
            y_predictions=pred_test,
            irrigation_train=irrigation_train,
            rain_train=rain_train,
            irrigation_test=irrigation_test,
            rain_test=rain_test)

In [70]:
file_path = os.path.join('predictions', 'sensor_68.pickle')

with open(file_path, 'wb') as file:
    pickle.dump(pred_sensor, file)

## Sensor 74

In [71]:
best_subset = ['62', '63', '70']

df_train = data[data['sensor_id'].isin(list(best_subset))]
df_test = data[data['sensor_id'] == '74']

X_train = df_train.drop(['sensor_id', 'avg_tens'], axis=1)
y_train = df_train['avg_tens']

X_test = df_test.drop(['sensor_id', 'avg_tens'], axis=1)
y_test = df_test['avg_tens']

selected_features = stepwise_bidirectional_selection(X_train, y_train, method='aic')

model = sm.OLS(y_train, sm.add_constant(X_train[selected_features])).fit()
pred_sensor = model.predict(sm.add_constant(X_test[selected_features]))


y_baseline = df_test['avg_tens_lag1'].reset_index(drop=True)

y_train_plot = y_train.reset_index(drop=True)
pred_train = model.predict(sm.add_constant(X_train[selected_features])).reset_index(drop=True)

y_test_plot = y_test.reset_index(drop=True)
pred_test = pred_sensor.reset_index(drop=True)

irrigation_train = df[df['sensor_id'].isin(list(best_subset))][['date', 'sum_irr']]
rain_train = df[df['sensor_id'].isin(list(best_subset))][['date', 'sum_rain']]

dates_to_remove = [datetime.date(2023, 4, 28), datetime.date(2023, 4, 29), datetime.date(2023, 4, 30)]
irrigation_train = irrigation_train[~irrigation_train['date'].isin(dates_to_remove)].reset_index(drop=True)
rain_train = rain_train[~rain_train['date'].isin(dates_to_remove)].reset_index(drop=True)

irrigation_train = irrigation_train['sum_irr']
rain_train = rain_train['sum_rain']

irrigation_test = df[df['sensor_id'] == '74'][['date', 'sum_irr']].reset_index(drop=True)
rain_test = df[df['sensor_id'] == '74'][['date', 'sum_rain']].reset_index(drop=True)

irrigation_test = irrigation_test[~irrigation_test['date'].isin(dates_to_remove)].reset_index(drop=True)
rain_test = rain_test[~rain_test['date'].isin(dates_to_remove)].reset_index(drop=True)

irrigation_test = irrigation_test['sum_irr']
rain_test = rain_test['sum_rain']


selected_features

In [72]:
show_figure(sensor_target_id=74,
            y_baseline=y_baseline,
            y_train=y_train_plot,
            y_pred=pred_train,
            y_test=y_test_plot,
            y_predictions=pred_test,
            irrigation_train=irrigation_train,
            rain_train=rain_train,
            irrigation_test=irrigation_test,
            rain_test=rain_test)

In [73]:
file_path = os.path.join('predictions', 'sensor_74.pickle')

with open(file_path, 'wb') as file:
    pickle.dump(pred_sensor, file)

## Sensor 75

In [74]:
best_subset = ['68', '73', '76']

df_train = data[data['sensor_id'].isin(list(best_subset))]
df_test = data[data['sensor_id'] == '75']

X_train = df_train.drop(['sensor_id', 'avg_tens'], axis=1)
y_train = df_train['avg_tens']

X_test = df_test.drop(['sensor_id', 'avg_tens'], axis=1)
y_test = df_test['avg_tens']

selected_features = stepwise_bidirectional_selection(X_train, y_train, method='aic')

model = sm.OLS(y_train, sm.add_constant(X_train[selected_features])).fit()
pred_sensor = model.predict(sm.add_constant(X_test[selected_features]))


y_baseline = df_test['avg_tens_lag1'].reset_index(drop=True)

y_train_plot = y_train.reset_index(drop=True)
pred_train = model.predict(sm.add_constant(X_train[selected_features])).reset_index(drop=True)

y_test_plot = y_test.reset_index(drop=True)
pred_test = pred_sensor.reset_index(drop=True)

irrigation_train = df[df['sensor_id'].isin(list(best_subset))][['date', 'sum_irr']]
rain_train = df[df['sensor_id'].isin(list(best_subset))][['date', 'sum_rain']]

dates_to_remove = [datetime.date(2023, 4, 28), datetime.date(2023, 4, 29), datetime.date(2023, 4, 30)]
irrigation_train = irrigation_train[~irrigation_train['date'].isin(dates_to_remove)].reset_index(drop=True)
rain_train = rain_train[~rain_train['date'].isin(dates_to_remove)].reset_index(drop=True)

irrigation_train = irrigation_train['sum_irr']
rain_train = rain_train['sum_rain']

irrigation_test = df[df['sensor_id'] == '75'][['date', 'sum_irr']].reset_index(drop=True)
rain_test = df[df['sensor_id'] == '75'][['date', 'sum_rain']].reset_index(drop=True)

irrigation_test = irrigation_test[~irrigation_test['date'].isin(dates_to_remove)].reset_index(drop=True)
rain_test = rain_test[~rain_test['date'].isin(dates_to_remove)].reset_index(drop=True)

irrigation_test = irrigation_test['sum_irr']
rain_test = rain_test['sum_rain']


selected_features

In [75]:
show_figure(sensor_target_id=75,
            y_baseline=y_baseline,
            y_train=y_train_plot,
            y_pred=pred_train,
            y_test=y_test_plot,
            y_predictions=pred_test,
            irrigation_train=irrigation_train,
            rain_train=rain_train,
            irrigation_test=irrigation_test,
            rain_test=rain_test)

In [76]:
file_path = os.path.join('predictions', 'sensor_75.pickle')

with open(file_path, 'wb') as file:
    pickle.dump(pred_sensor, file)

# Other Sensors

## Sensor 65

In [86]:
all_sensors = ['66', '69', '73', '76']
all_subsets = list(chain.from_iterable(combinations(all_sensors, r) for r in range(1, len(all_sensors) + 1)))

df_test = data[data['sensor_id'] == '65']
y_baseline = df_test['avg_tens_lag1'].reset_index(drop=True)

X_train = data[data['sensor_id'].isin(list(all_sensors))]
ids_train = X_train[['sensor_id']].reset_index(drop=True)
y_train = X_train[['avg_tens']].reset_index(drop=True)
X_train = X_train.drop(columns=['sensor_id', 'avg_tens'])

X_test = data[data['sensor_id'] == '65']
ids_test = X_test[['sensor_id']].reset_index(drop=True)
y_test = X_test[['avg_tens']].reset_index(drop=True)
X_test = X_test.drop(columns=['sensor_id', 'avg_tens'])

best_n_comp = optimize_ncomp_pls(X_train, y_train, 50, 'RMSE', 'min')

pls_model = PLSRegression(n_components=best_n_comp)
pls_model.fit(X_train, y_train)


pls_labels = [f'PC{i}' for i in range(1, best_n_comp + 1)]
pls_model = PLSRegression(n_components=best_n_comp)
pls_model.fit(X_train, y_train)

X_train_pls = pls_model.transform(X_train)
X_train_pls = pd.DataFrame(X_train_pls, columns=pls_labels)
X_test_pls = pls_model.transform(X_test)
X_test_pls = pd.DataFrame(X_test_pls, columns=pls_labels)

train_pls = pd.concat([ids_train, y_train, X_train_pls], axis=1)
test_pls = pd.concat([ids_test, y_test, X_test_pls], axis=1)
data_pls = pd.concat([train_pls, test_pls], ignore_index=True)


best_subset = ['66']

df_train = data_pls[data_pls['sensor_id'].isin(list(best_subset))]
df_test = data_pls[data_pls['sensor_id'] == '65']

X_train = df_train.drop(['sensor_id', 'avg_tens'], axis=1)
y_train = df_train['avg_tens']

X_test = df_test.drop(['sensor_id', 'avg_tens'], axis=1)
y_test = df_test['avg_tens']


model = sm.OLS(y_train, sm.add_constant(X_train)).fit()
pred_sensor = model.predict(sm.add_constant(X_test))


y_train_plot = y_train.reset_index(drop=True)
pred_train = model.predict(sm.add_constant(X_train)).reset_index(drop=True)

y_test_plot = y_test.reset_index(drop=True)
pred_test = pred_sensor.reset_index(drop=True)

irrigation_train = df[df['sensor_id'].isin(list(best_subset))][['date', 'sum_irr']]
rain_train = df[df['sensor_id'].isin(list(best_subset))][['date', 'sum_rain']]

dates_to_remove = [datetime.date(2023, 4, 28), datetime.date(2023, 4, 29), datetime.date(2023, 4, 30)]
irrigation_train = irrigation_train[~irrigation_train['date'].isin(dates_to_remove)].reset_index(drop=True)
rain_train = rain_train[~rain_train['date'].isin(dates_to_remove)].reset_index(drop=True)

irrigation_train = irrigation_train['sum_irr']
rain_train = rain_train['sum_rain']

irrigation_test = df[df['sensor_id'] == '65'][['date', 'sum_irr']].reset_index(drop=True)
rain_test = df[df['sensor_id'] == '65'][['date', 'sum_rain']].reset_index(drop=True)

irrigation_test = irrigation_test[~irrigation_test['date'].isin(dates_to_remove)].reset_index(drop=True)
rain_test = rain_test[~rain_test['date'].isin(dates_to_remove)].reset_index(drop=True)

irrigation_test = irrigation_test['sum_irr']
rain_test = rain_test['sum_rain']


pls_model

In [79]:
r_squared_arimax = round(r_squared(y_test, pred_sensor), 3)
mape_arimax = round(mape(y_test, pred_sensor), 3)
mape_std_arimax = round(mape_std(y_test, pred_sensor), 3)
mae_arimax = round(mae(y_test, pred_sensor), 3)
rmse_arimax = round(rmse(y_test, pred_sensor), 3)
rmse_std_arimax = round(rmse_std(y_test, pred_sensor), 3)


table = [
    ['Metric', 'ARIMAX'],
    ['R-squared', r_squared_arimax],
    ['MAPE', mape_arimax],
    ['MAPE_std', mape_std_arimax],
    ['MAE', mae_arimax],
    ['RMSE', rmse_arimax],
    ['RMSE_std', rmse_std_arimax]
]

print(tabulate(table, headers='firstrow', tablefmt='fancy_grid'))

In [80]:
show_figure(sensor_target_id=65,
            y_baseline=y_baseline,
            y_train=y_train_plot,
            y_pred=pred_train,
            y_test=y_test_plot,
            y_predictions=pred_test,
            irrigation_train=irrigation_train,
            rain_train=rain_train,
            irrigation_test=irrigation_test,
            rain_test=rain_test)

In [81]:
file_path = os.path.join('predictions', 'sensor_65.pickle')

with open(file_path, 'wb') as file:
    pickle.dump(pred_sensor, file)

## Sensor 66

In [82]:
best_subset = ['65', '76']

df_train = data[data['sensor_id'].isin(list(best_subset))]
df_test = data[data['sensor_id'] == '66']

X_train = df_train.drop(['sensor_id', 'avg_tens'], axis=1)
y_train = df_train['avg_tens']

X_test = df_test.drop(['sensor_id', 'avg_tens'], axis=1)
y_test = df_test['avg_tens']

selected_features = stepwise_bidirectional_selection(X_train, y_train, method='aic')

model = sm.OLS(y_train, sm.add_constant(X_train[selected_features])).fit()
pred_sensor = model.predict(sm.add_constant(X_test[selected_features]))


y_baseline = df_test['avg_tens_lag1'].reset_index(drop=True)
y_train_plot = y_train.reset_index(drop=True)
pred_train = model.predict(sm.add_constant(X_train[selected_features])).reset_index(drop=True)

y_test_plot = y_test.reset_index(drop=True)
pred_test = pred_sensor.reset_index(drop=True)

irrigation_train = df[df['sensor_id'].isin(list(best_subset))][['date', 'sum_irr']]
rain_train = df[df['sensor_id'].isin(list(best_subset))][['date', 'sum_rain']]

dates_to_remove = [datetime.date(2023, 4, 28), datetime.date(2023, 4, 29), datetime.date(2023, 4, 30)]
irrigation_train = irrigation_train[~irrigation_train['date'].isin(dates_to_remove)].reset_index(drop=True)
rain_train = rain_train[~rain_train['date'].isin(dates_to_remove)].reset_index(drop=True)

irrigation_train = irrigation_train['sum_irr']
rain_train = rain_train['sum_rain']

irrigation_test = df[df['sensor_id'] == '66'][['date', 'sum_irr']].reset_index(drop=True)
rain_test = df[df['sensor_id'] == '66'][['date', 'sum_rain']].reset_index(drop=True)

irrigation_test = irrigation_test[~irrigation_test['date'].isin(dates_to_remove)].reset_index(drop=True)
rain_test = rain_test[~rain_test['date'].isin(dates_to_remove)].reset_index(drop=True)

irrigation_test = irrigation_test['sum_irr']
rain_test = rain_test['sum_rain']


selected_features

In [83]:
r_squared_arimax = round(r_squared(y_test, pred_sensor), 3)
mape_arimax = round(mape(y_test, pred_sensor), 3)
mape_std_arimax = round(mape_std(y_test, pred_sensor), 3)
mae_arimax = round(mae(y_test, pred_sensor), 3)
rmse_arimax = round(rmse(y_test, pred_sensor), 3)
rmse_std_arimax = round(rmse_std(y_test, pred_sensor), 3)


table = [
    ['Metric', 'ARIMAX'],
    ['R-squared', r_squared_arimax],
    ['MAPE', mape_arimax],
    ['MAPE_std', mape_std_arimax],
    ['MAE', mae_arimax],
    ['RMSE', rmse_arimax],
    ['RMSE_std', rmse_std_arimax]
]

print(tabulate(table, headers='firstrow', tablefmt='fancy_grid'))

In [84]:
show_figure(sensor_target_id=66,
            y_baseline=y_baseline,
            y_train=y_train_plot,
            y_pred=pred_train,
            y_test=y_test_plot,
            y_predictions=pred_test,
            irrigation_train=irrigation_train,
            rain_train=rain_train,
            irrigation_test=irrigation_test,
            rain_test=rain_test)

In [85]:
file_path = os.path.join('predictions', 'sensor_66.pickle')

with open(file_path, 'wb') as file:
    pickle.dump(pred_sensor, file)

## Sensor 69

In [87]:
all_sensors = ['65', '66', '73', '76']
all_subsets = list(chain.from_iterable(combinations(all_sensors, r) for r in range(1, len(all_sensors) + 1)))

df_test = data[data['sensor_id'] == '69']
y_baseline = df_test['avg_tens_lag1'].reset_index(drop=True)

X_train = data[data['sensor_id'].isin(list(all_sensors))]
ids_train = X_train[['sensor_id']].reset_index(drop=True)
y_train = X_train[['avg_tens']].reset_index(drop=True)
X_train = X_train.drop(columns=['sensor_id', 'avg_tens'])

X_test = data[data['sensor_id'] == '69']
ids_test = X_test[['sensor_id']].reset_index(drop=True)
y_test = X_test[['avg_tens']].reset_index(drop=True)
X_test = X_test.drop(columns=['sensor_id', 'avg_tens'])

best_n_comp = optimize_ncomp_pls(X_train, y_train, 50, 'RMSE', 'min')

pls_model = PLSRegression(n_components=best_n_comp)
pls_model.fit(X_train, y_train)


pls_labels = [f'PC{i}' for i in range(1, best_n_comp + 1)]
pls_model = PLSRegression(n_components=best_n_comp)
pls_model.fit(X_train, y_train)

X_train_pls = pls_model.transform(X_train)
X_train_pls = pd.DataFrame(X_train_pls, columns=pls_labels)
X_test_pls = pls_model.transform(X_test)
X_test_pls = pd.DataFrame(X_test_pls, columns=pls_labels)

train_pls = pd.concat([ids_train, y_train, X_train_pls], axis=1)
test_pls = pd.concat([ids_test, y_test, X_test_pls], axis=1)
data_pls = pd.concat([train_pls, test_pls], ignore_index=True)


best_subset = ['65', '76']

df_train = data_pls[data_pls['sensor_id'].isin(list(best_subset))]
df_test = data_pls[data_pls['sensor_id'] == '69']

X_train = df_train.drop(['sensor_id', 'avg_tens'], axis=1)
y_train = df_train['avg_tens']

X_test = df_test.drop(['sensor_id', 'avg_tens'], axis=1)
y_test = df_test['avg_tens']


model = sm.OLS(y_train, sm.add_constant(X_train)).fit()
pred_sensor = model.predict(sm.add_constant(X_test))


y_train_plot = y_train.reset_index(drop=True)
pred_train = model.predict(sm.add_constant(X_train)).reset_index(drop=True)

y_test_plot = y_test.reset_index(drop=True)
pred_test = pred_sensor.reset_index(drop=True)

irrigation_train = df[df['sensor_id'].isin(list(best_subset))][['date', 'sum_irr']]
rain_train = df[df['sensor_id'].isin(list(best_subset))][['date', 'sum_rain']]

dates_to_remove = [datetime.date(2023, 4, 28), datetime.date(2023, 4, 29), datetime.date(2023, 4, 30)]
irrigation_train = irrigation_train[~irrigation_train['date'].isin(dates_to_remove)].reset_index(drop=True)
rain_train = rain_train[~rain_train['date'].isin(dates_to_remove)].reset_index(drop=True)

irrigation_train = irrigation_train['sum_irr']
rain_train = rain_train['sum_rain']

irrigation_test = df[df['sensor_id'] == '69'][['date', 'sum_irr']].reset_index(drop=True)
rain_test = df[df['sensor_id'] == '69'][['date', 'sum_rain']].reset_index(drop=True)

irrigation_test = irrigation_test[~irrigation_test['date'].isin(dates_to_remove)].reset_index(drop=True)
rain_test = rain_test[~rain_test['date'].isin(dates_to_remove)].reset_index(drop=True)

irrigation_test = irrigation_test['sum_irr']
rain_test = rain_test['sum_rain']


pls_model

In [88]:
r_squared_arimax = round(r_squared(y_test, pred_sensor), 3)
mape_arimax = round(mape(y_test, pred_sensor), 3)
mape_std_arimax = round(mape_std(y_test, pred_sensor), 3)
mae_arimax = round(mae(y_test, pred_sensor), 3)
rmse_arimax = round(rmse(y_test, pred_sensor), 3)
rmse_std_arimax = round(rmse_std(y_test, pred_sensor), 3)


table = [
    ['Metric', 'ARIMAX'],
    ['R-squared', r_squared_arimax],
    ['MAPE', mape_arimax],
    ['MAPE_std', mape_std_arimax],
    ['MAE', mae_arimax],
    ['RMSE', rmse_arimax],
    ['RMSE_std', rmse_std_arimax]
]

print(tabulate(table, headers='firstrow', tablefmt='fancy_grid'))

In [89]:
show_figure(sensor_target_id=69,
            y_baseline=y_baseline,
            y_train=y_train_plot,
            y_pred=pred_train,
            y_test=y_test_plot,
            y_predictions=pred_test,
            irrigation_train=irrigation_train,
            rain_train=rain_train,
            irrigation_test=irrigation_test,
            rain_test=rain_test)

In [90]:
file_path = os.path.join('predictions', 'sensor_69.pickle')

with open(file_path, 'wb') as file:
    pickle.dump(pred_sensor, file)

## Sensor 71

In [91]:
best_subset = ['61', '64', '70']

df_train = data[data['sensor_id'].isin(list(best_subset))]
df_test = data[data['sensor_id'] == '71']

X_train = df_train.drop(['sensor_id', 'avg_tens'], axis=1)
y_train = df_train['avg_tens']

X_test = df_test.drop(['sensor_id', 'avg_tens'], axis=1)
y_test = df_test['avg_tens']

selected_features = stepwise_bidirectional_selection(X_train, y_train, method='aic')

model = sm.OLS(y_train, sm.add_constant(X_train[selected_features])).fit()
pred_sensor = model.predict(sm.add_constant(X_test[selected_features]))


y_baseline = df_test['avg_tens_lag1'].reset_index(drop=True)
y_train_plot = y_train.reset_index(drop=True)
pred_train = model.predict(sm.add_constant(X_train[selected_features])).reset_index(drop=True)

y_test_plot = y_test.reset_index(drop=True)
pred_test = pred_sensor.reset_index(drop=True)

irrigation_train = df[df['sensor_id'].isin(list(best_subset))][['date', 'sum_irr']]
rain_train = df[df['sensor_id'].isin(list(best_subset))][['date', 'sum_rain']]

dates_to_remove = [datetime.date(2023, 4, 28), datetime.date(2023, 4, 29), datetime.date(2023, 4, 30)]
irrigation_train = irrigation_train[~irrigation_train['date'].isin(dates_to_remove)].reset_index(drop=True)
rain_train = rain_train[~rain_train['date'].isin(dates_to_remove)].reset_index(drop=True)

irrigation_train = irrigation_train['sum_irr']
rain_train = rain_train['sum_rain']

irrigation_test = df[df['sensor_id'] == '71'][['date', 'sum_irr']].reset_index(drop=True)
rain_test = df[df['sensor_id'] == '71'][['date', 'sum_rain']].reset_index(drop=True)

irrigation_test = irrigation_test[~irrigation_test['date'].isin(dates_to_remove)].reset_index(drop=True)
rain_test = rain_test[~rain_test['date'].isin(dates_to_remove)].reset_index(drop=True)

irrigation_test = irrigation_test['sum_irr']
rain_test = rain_test['sum_rain']


selected_features

In [92]:
r_squared_arimax = round(r_squared(y_test, pred_sensor), 3)
mape_arimax = round(mape(y_test, pred_sensor), 3)
mape_std_arimax = round(mape_std(y_test, pred_sensor), 3)
mae_arimax = round(mae(y_test, pred_sensor), 3)
rmse_arimax = round(rmse(y_test, pred_sensor), 3)
rmse_std_arimax = round(rmse_std(y_test, pred_sensor), 3)


table = [
    ['Metric', 'ARIMAX'],
    ['R-squared', r_squared_arimax],
    ['MAPE', mape_arimax],
    ['MAPE_std', mape_std_arimax],
    ['MAE', mae_arimax],
    ['RMSE', rmse_arimax],
    ['RMSE_std', rmse_std_arimax]
]

print(tabulate(table, headers='firstrow', tablefmt='fancy_grid'))

In [93]:
show_figure(sensor_target_id=71,
            y_baseline=y_baseline,
            y_train=y_train_plot,
            y_pred=pred_train,
            y_test=y_test_plot,
            y_predictions=pred_test,
            irrigation_train=irrigation_train,
            rain_train=rain_train,
            irrigation_test=irrigation_test,
            rain_test=rain_test)

In [94]:
file_path = os.path.join('predictions', 'sensor_71.pickle')

with open(file_path, 'wb') as file:
    pickle.dump(pred_sensor, file)

## Sensor 72

In [95]:
best_subset = ['62', '67', '68', '71']

df_train = data[data['sensor_id'].isin(list(best_subset))]
df_test = data[data['sensor_id'] == '72']

X_train = df_train.drop(['sensor_id', 'avg_tens'], axis=1)
y_train = df_train['avg_tens']

X_test = df_test.drop(['sensor_id', 'avg_tens'], axis=1)
y_test = df_test['avg_tens']

selected_features = stepwise_bidirectional_selection(X_train, y_train, method='aic')

model = sm.OLS(y_train, sm.add_constant(X_train[selected_features])).fit()
pred_sensor = model.predict(sm.add_constant(X_test[selected_features]))


y_baseline = df_test['avg_tens_lag1'].reset_index(drop=True)
y_train_plot = y_train.reset_index(drop=True)
pred_train = model.predict(sm.add_constant(X_train[selected_features])).reset_index(drop=True)

y_test_plot = y_test.reset_index(drop=True)
pred_test = pred_sensor.reset_index(drop=True)

irrigation_train = df[df['sensor_id'].isin(list(best_subset))][['date', 'sum_irr']]
rain_train = df[df['sensor_id'].isin(list(best_subset))][['date', 'sum_rain']]

dates_to_remove = [datetime.date(2023, 4, 28), datetime.date(2023, 4, 29), datetime.date(2023, 4, 30)]
irrigation_train = irrigation_train[~irrigation_train['date'].isin(dates_to_remove)].reset_index(drop=True)
rain_train = rain_train[~rain_train['date'].isin(dates_to_remove)].reset_index(drop=True)

irrigation_train = irrigation_train['sum_irr']
rain_train = rain_train['sum_rain']

irrigation_test = df[df['sensor_id'] == '72'][['date', 'sum_irr']].reset_index(drop=True)
rain_test = df[df['sensor_id'] == '72'][['date', 'sum_rain']].reset_index(drop=True)

irrigation_test = irrigation_test[~irrigation_test['date'].isin(dates_to_remove)].reset_index(drop=True)
rain_test = rain_test[~rain_test['date'].isin(dates_to_remove)].reset_index(drop=True)

irrigation_test = irrigation_test['sum_irr']
rain_test = rain_test['sum_rain']

selected_features

In [96]:
r_squared_arimax = round(r_squared(y_test, pred_sensor), 3)
mape_arimax = round(mape(y_test, pred_sensor), 3)
mape_std_arimax = round(mape_std(y_test, pred_sensor), 3)
mae_arimax = round(mae(y_test, pred_sensor), 3)
rmse_arimax = round(rmse(y_test, pred_sensor), 3)
rmse_std_arimax = round(rmse_std(y_test, pred_sensor), 3)


table = [
    ['Metric', 'ARIMAX'],
    ['R-squared', r_squared_arimax],
    ['MAPE', mape_arimax],
    ['MAPE_std', mape_std_arimax],
    ['MAE', mae_arimax],
    ['RMSE', rmse_arimax],
    ['RMSE_std', rmse_std_arimax]
]

print(tabulate(table, headers='firstrow', tablefmt='fancy_grid'))

In [97]:
show_figure(sensor_target_id=72,
            y_baseline=y_baseline,
            y_train=y_train_plot,
            y_pred=pred_train,
            y_test=y_test_plot,
            y_predictions=pred_test,
            irrigation_train=irrigation_train,
            rain_train=rain_train,
            irrigation_test=irrigation_test,
            rain_test=rain_test)

In [98]:
file_path = os.path.join('predictions', 'sensor_72.pickle')

with open(file_path, 'wb') as file:
    pickle.dump(pred_sensor, file)

## Sensor 73

In [99]:
best_subset = ['65', '66', '75', '76']

df_train = data[data['sensor_id'].isin(list(best_subset))]
df_test = data[data['sensor_id'] == '73']

X_train = df_train.drop(['sensor_id', 'avg_tens'], axis=1)
y_train = df_train['avg_tens']

X_test = df_test.drop(['sensor_id', 'avg_tens'], axis=1)
y_test = df_test['avg_tens']

selected_features = stepwise_bidirectional_selection(X_train, y_train, method='aic')

model = sm.OLS(y_train, sm.add_constant(X_train[selected_features])).fit()
pred_sensor = model.predict(sm.add_constant(X_test[selected_features]))


y_baseline = df_test['avg_tens_lag1'].reset_index(drop=True)
y_train_plot = y_train.reset_index(drop=True)
pred_train = model.predict(sm.add_constant(X_train[selected_features])).reset_index(drop=True)

y_test_plot = y_test.reset_index(drop=True)
pred_test = pred_sensor.reset_index(drop=True)

irrigation_train = df[df['sensor_id'].isin(list(best_subset))][['date', 'sum_irr']]
rain_train = df[df['sensor_id'].isin(list(best_subset))][['date', 'sum_rain']]

dates_to_remove = [datetime.date(2023, 4, 28), datetime.date(2023, 4, 29), datetime.date(2023, 4, 30)]
irrigation_train = irrigation_train[~irrigation_train['date'].isin(dates_to_remove)].reset_index(drop=True)
rain_train = rain_train[~rain_train['date'].isin(dates_to_remove)].reset_index(drop=True)

irrigation_train = irrigation_train['sum_irr']
rain_train = rain_train['sum_rain']

irrigation_test = df[df['sensor_id'] == '73'][['date', 'sum_irr']].reset_index(drop=True)
rain_test = df[df['sensor_id'] == '73'][['date', 'sum_rain']].reset_index(drop=True)

irrigation_test = irrigation_test[~irrigation_test['date'].isin(dates_to_remove)].reset_index(drop=True)
rain_test = rain_test[~rain_test['date'].isin(dates_to_remove)].reset_index(drop=True)

irrigation_test = irrigation_test['sum_irr']
rain_test = rain_test['sum_rain']


selected_features

In [101]:
r_squared_arimax = round(r_squared(y_test, pred_sensor), 3)
mape_arimax = round(mape(y_test, pred_sensor), 3)
mape_std_arimax = round(mape_std(y_test, pred_sensor), 3)
mae_arimax = round(mae(y_test, pred_sensor), 3)
rmse_arimax = round(rmse(y_test, pred_sensor), 3)
rmse_std_arimax = round(rmse_std(y_test, pred_sensor), 3)


table = [
    ['Metric', 'ARIMAX'],
    ['R-squared', r_squared_arimax],
    ['MAPE', mape_arimax],
    ['MAPE_std', mape_std_arimax],
    ['MAE', mae_arimax],
    ['RMSE', rmse_arimax],
    ['RMSE_std', rmse_std_arimax]
]

print(tabulate(table, headers='firstrow', tablefmt='fancy_grid'))

In [102]:
show_figure(sensor_target_id=73,
            y_baseline=y_baseline,
            y_train=y_train_plot,
            y_pred=pred_train,
            y_test=y_test_plot,
            y_predictions=pred_test,
            irrigation_train=irrigation_train,
            rain_train=rain_train,
            irrigation_test=irrigation_test,
            rain_test=rain_test)

In [103]:
file_path = os.path.join('predictions', 'sensor_73.pickle')

with open(file_path, 'wb') as file:
    pickle.dump(pred_sensor, file)

## Sensor 76

In [104]:
all_sensors = ['65', '66', '69', '73']
all_subsets = list(chain.from_iterable(combinations(all_sensors, r) for r in range(1, len(all_sensors) + 1)))

df_test = data[data['sensor_id'] == '76']
y_baseline = df_test['avg_tens_lag1'].reset_index(drop=True)

X_train = data[data['sensor_id'].isin(list(all_sensors))]
ids_train = X_train[['sensor_id']].reset_index(drop=True)
y_train = X_train[['avg_tens']].reset_index(drop=True)
X_train = X_train.drop(columns=['sensor_id', 'avg_tens'])

X_test = data[data['sensor_id'] == '76']
ids_test = X_test[['sensor_id']].reset_index(drop=True)
y_test = X_test[['avg_tens']].reset_index(drop=True)
X_test = X_test.drop(columns=['sensor_id', 'avg_tens'])

best_n_comp = optimize_ncomp_pls(X_train, y_train, 50, 'RMSE', 'min')

pls_model = PLSRegression(n_components=best_n_comp)
pls_model.fit(X_train, y_train)


pls_labels = [f'PC{i}' for i in range(1, best_n_comp + 1)]
pls_model = PLSRegression(n_components=best_n_comp)
pls_model.fit(X_train, y_train)

X_train_pls = pls_model.transform(X_train)
X_train_pls = pd.DataFrame(X_train_pls, columns=pls_labels)
X_test_pls = pls_model.transform(X_test)
X_test_pls = pd.DataFrame(X_test_pls, columns=pls_labels)

train_pls = pd.concat([ids_train, y_train, X_train_pls], axis=1)
test_pls = pd.concat([ids_test, y_test, X_test_pls], axis=1)
data_pls = pd.concat([train_pls, test_pls], ignore_index=True)


best_subset = ['66', '69', '73']

df_train = data_pls[data_pls['sensor_id'].isin(list(best_subset))]
df_test = data_pls[data_pls['sensor_id'] == '76']

X_train = df_train.drop(['sensor_id', 'avg_tens'], axis=1)
y_train = df_train['avg_tens']

X_test = df_test.drop(['sensor_id', 'avg_tens'], axis=1)
y_test = df_test['avg_tens']


model = sm.OLS(y_train, sm.add_constant(X_train)).fit()
pred_sensor = model.predict(sm.add_constant(X_test))


y_train_plot = y_train.reset_index(drop=True)
pred_train = model.predict(sm.add_constant(X_train)).reset_index(drop=True)

y_test_plot = y_test.reset_index(drop=True)
pred_test = pred_sensor.reset_index(drop=True)

irrigation_train = df[df['sensor_id'].isin(list(best_subset))][['date', 'sum_irr']]
rain_train = df[df['sensor_id'].isin(list(best_subset))][['date', 'sum_rain']]

dates_to_remove = [datetime.date(2023, 4, 28), datetime.date(2023, 4, 29), datetime.date(2023, 4, 30)]
irrigation_train = irrigation_train[~irrigation_train['date'].isin(dates_to_remove)].reset_index(drop=True)
rain_train = rain_train[~rain_train['date'].isin(dates_to_remove)].reset_index(drop=True)

irrigation_train = irrigation_train['sum_irr']
rain_train = rain_train['sum_rain']

irrigation_test = df[df['sensor_id'] == '69'][['date', 'sum_irr']].reset_index(drop=True)
rain_test = df[df['sensor_id'] == '69'][['date', 'sum_rain']].reset_index(drop=True)

irrigation_test = irrigation_test[~irrigation_test['date'].isin(dates_to_remove)].reset_index(drop=True)
rain_test = rain_test[~rain_test['date'].isin(dates_to_remove)].reset_index(drop=True)

irrigation_test = irrigation_test['sum_irr']
rain_test = rain_test['sum_rain']


pls_model

In [105]:
r_squared_arimax = round(r_squared(y_test, pred_sensor), 3)
mape_arimax = round(mape(y_test, pred_sensor), 3)
mape_std_arimax = round(mape_std(y_test, pred_sensor), 3)
mae_arimax = round(mae(y_test, pred_sensor), 3)
rmse_arimax = round(rmse(y_test, pred_sensor), 3)
rmse_std_arimax = round(rmse_std(y_test, pred_sensor), 3)


table = [
    ['Metric', 'ARIMAX'],
    ['R-squared', r_squared_arimax],
    ['MAPE', mape_arimax],
    ['MAPE_std', mape_std_arimax],
    ['MAE', mae_arimax],
    ['RMSE', rmse_arimax],
    ['RMSE_std', rmse_std_arimax]
]

print(tabulate(table, headers='firstrow', tablefmt='fancy_grid'))

In [106]:
show_figure(sensor_target_id=76,
            y_baseline=y_baseline,
            y_train=y_train_plot,
            y_pred=pred_train,
            y_test=y_test_plot,
            y_predictions=pred_test,
            irrigation_train=irrigation_train,
            rain_train=rain_train,
            irrigation_test=irrigation_test,
            rain_test=rain_test)

In [107]:
file_path = os.path.join('predictions', 'sensor_76.pickle')

with open(file_path, 'wb') as file:
    pickle.dump(pred_sensor, file)